In [1]:
from __future__ import annotations

from typing import Union

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
#plt.style.use('/home/inca/default.mplstyle')

In [5]:
players_2324_path = "./Quotazioni_Fantacalcio_Stagione_2023_24.csv"
players_2223_path = "./Quotazioni_Fantacalcio_Stagione_2022_23.csv"
stats_2223_path = "./Statistiche_Fantacalcio_Stagione_2022_23.csv"
stats_2324_path = "./Statistiche_Fantacalcio_Stagione_2023_24.csv"
# the latter is updated at the third round - 3rd Sep 2023

players_2324_df = pd.read_csv(players_2324_path, delimiter=",", header=1, index_col=False)
players_2223_df = pd.read_csv(players_2223_path, delimiter=",", header=1, index_col=False)
stats_2223_df = pd.read_csv(stats_2223_path, delimiter=",", header=1, index_col=False)
stats_2324_df = pd.read_csv(stats_2324_path, delimiter=",", header=1, index_col=False)

# basic functions

In [4]:
class Filter:

    def __init__(
            self: Filter, column: str, operation: str, value: Union[str, float]) -> None:
        self.column = column
        self.operation = operation
        self.value = value
    
    def apply_to_df(self: Filter, df: pd.DataFrame) -> pd.DataFrame:
        if self.operation == "ge":
            return df[df[self.column] > self.value]
        elif self.operation == "geq":
            return df[df[self.column] >= self.value]
        elif self.operation == "le":
            return df[df[self.column] < self.value]
        elif self.operation == "leq":
            return df[df[self.column] <= self.value]
        elif self.operation == "eq":
            return df[df[self.column] == self.value]
        elif self.operation == "neq":
            return df[df[self.column] != self.value]


def apply_multiple_filters(df: pd.DataFrame, filter_list: list[Filter]) -> pd.DataFrame:
    filtered_df = df.copy()
    for filter in filter_list:
        filtered_df = filter.apply_to_df(filtered_df)
    return filtered_df


def extract_players_mantra_position(df: pd.DataFrame, position: str) -> pd.DataFrame:
    """
    The mantra position is tricky, because they actually include multiple positions with ; as delimiter.
    The easiest way is to select the players where the mantra position column contains also that specific position.
    """
    return df[df["RM"].str.contains(position)]


In [24]:
# example: extract players of a team
Filter("Squadra", "eq", "Verona").apply_to_df(players_2324_df)

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
15,4957,P,Por,Montipo',Verona,8,7,1,8,7,1,13,13
33,511,P,Por,Perilli,Verona,1,1,0,1,1,0,1,1
49,4491,P,Por,Berardi A.,Verona,1,1,0,1,1,0,1,1
84,5851,D,Ds;E,Doig,Verona,11,11,0,11,11,0,20,20
97,581,D,Dd;E,Faraoni,Verona,9,9,0,9,9,0,16,16
128,6046,D,Dc,Hien,Verona,7,8,-1,7,8,-1,15,15
165,4493,D,Dc,Dawidowicz,Verona,5,4,1,5,4,1,5,5
183,5812,D,Dd;E,Terracciano F.,Verona,4,3,1,4,3,1,7,7
194,2758,D,Dc,Gunter,Verona,3,4,-1,3,4,-1,4,4
195,2769,D,Dc,Magnani,Verona,3,3,0,3,3,0,4,4


In [7]:
# example on how to use a filter
filter = Filter("Qt.A M", "geq", 15)
good_players = filter.apply_to_df(players_2324_df)
good_players

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
0,4312,P,Por,Maignan,Milan,17,17,0,17,17,0,89,89
1,572,P,Por,Meret,Napoli,16,17,-1,16,17,-1,80,80
2,2428,P,Por,Sommer,Inter,16,15,1,16,15,1,67,67
3,453,P,Por,Szczesny,Juventus,15,15,0,15,15,0,71,71
4,2814,P,Por,Provedel,Lazio,15,16,-1,15,16,-1,60,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,2002,A,A,Chiesa,Juventus,19,17,2,19,17,2,142,142
458,2012,A,Pc,Milik,Juventus,18,18,0,17,17,0,92,82
459,4179,A,A,Gonzalez N.,Fiorentina,17,15,2,17,15,2,95,100
460,6227,A,Pc,Okafor,Milan,16,16,0,15,15,0,52,48


In [8]:
# example on how to use multiple filters to extract cheap but good goalkeepers
filter_list = [
    Filter("RM", "eq", "Por"),
    Filter("Qt.A M", "geq", 10),
    Filter("FVM M", "leq", 50)
]
filtered_df = apply_multiple_filters(players_2324_df, filter_list)
filtered_df

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
5,4270,P,Por,Rui Patricio,Roma,12,14,-2,12,14,-2,47,47
6,5876,P,Por,Di Gregorio,Monza,12,12,0,12,12,0,37,37
7,133,P,Por,Skorupski,Bologna,10,10,0,10,10,0,32,32
8,2170,P,Por,Milinkovic-Savic V.,Torino,10,10,0,10,10,0,31,31
9,2815,P,Por,Terracciano,Fiorentina,10,10,0,10,10,0,14,14
10,4431,P,Por,Carnesecchi,Atalanta,10,11,-1,10,11,-1,10,16


In [9]:
# example: players who can play also as difensori centrali
difensori_centrali = extract_players_mantra_position(players_2324_df, "Dc")
difensori_centrali

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
66,4237,D,Dd;Ds;Dc,Danilo,Juventus,16,15,1,19,18,1,52,48
68,6041,D,Dc,Schuurs,Torino,15,14,1,15,14,1,46,46
69,6066,D,Dd;Dc,Posch,Bologna,15,16,-1,17,18,-1,46,50
70,2120,D,Dc,Bastoni,Inter,14,13,1,14,13,1,48,48
71,4177,D,Dd;Dc,Pavard,Inter,14,14,0,15,15,0,48,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,6411,D,Dc,Mateus Lusuardi,Frosinone,1,1,0,1,1,0,1,1
249,6430,D,Dc,Kalaj,Frosinone,1,1,0,1,1,0,1,1
251,6433,D,Dc,Huijsen,Juventus,1,1,0,1,1,0,1,1
252,6444,D,Dc,Bonfanti,Atalanta,1,1,0,1,1,0,1,1


# pre-asta: interesting players

In [25]:
atalanta_players = ["Lookman", "Scamacca", "Zapata D.", "Scalvini",
                    "Toure' E.", "Zappacosta", "Kolasinac", "Bakker",
                    "Ederson D.s.", "Koopmeiners", "De Roon", "De Ketelaere"]

bologna_players = ["Zirkzee", "Beukema", "Ferguson", "Moro N.", "Orsolini", "Dominguez"]

cagliari_players = ["Lapadula", "Jankto", "Sulemana I.", "Makoumbou", "Luvumbo", "Nandez"]

empoli_players = ["Baldanzi", "Gyasi", "Caputo", "Henderson", "Cancellieri", "Shpendi"]

fiorentina_players = ["Amrabat", "Gonzalez N.", "Arthur Melo", "Milenkovic",
                      "Bonaventura", "Dodo'", "Mina", "Parisi", "Kayode", "Beltran"]

frosinone_players = ["Cuni", "Caso"]

genoa_players = ["Malinovskyi", "Dragusin", "Retegui", "Strootman", "Gudmundsson A.", "Messias", "Aramu"]

inter_players = ["Martinez L.", "Barella", "Dimarco", "Bastoni", "Frattesi", "Carlos Augusto"]

juventus_players = ["Danilo", "Weah", "Fagioli", "Miretti", "Kostic", "Locatelli", "Chiesa"]

lazio_players = ["Rovella", "Casale", "Provedel", "Lazzari", "Zaccagni", "Luis Alberto", "Kamada", "Immobile"]

lecce_players = ["Rafia", "Maleh", "Baschirotto", "Almqvist", "Strefezza", "Krstovic"]

milan_players = ["Chukwueze", "Maignan", "Hernandez T.", "Thiaw", "Kalulu",
                 "Reijnders", "Romero L.", "Krunic", "Rafael Leao", "Bennacer", "Loftus-Cheek"]

monza_players = ["Ciurria", "Di Gregorio", "Colpani", "Pessina", "Mari'", "D'ambrosio", "Colombo"]

napoli_players = ["Rrahmani", "Cajuste", "Zambo Anguissa", "Simeone",
                  "Raspadori", "Di Lorenzo", "Meret", "Mario Rui"]

roma_players = ["Pellegrini Lo.", "Renato Sanches", "Zalewski", "Bove", "Smalling", "N'dicka", "Aouar"]

salernitana_players = ["Mazzocchi", "Candreva", "Ochoa", "Coulibaly L.", "Kastanos"]

sassuolo_players = ["Berardi", "Lauriente'", "Lopez M.", "Matheus Henrique", "Viti", "Mulattieri", "Erlic", "Bajrami"]

torino_players = ["Milinkovic-Savic V.", "Vlasic", "Sanabria", "Bellanova",
                  "Schuurs", "Ilic", "Vojvoda", "Ricci S."]

udinese_players = ["Deulofeu", "Beto", "Kamara H.", "Lovric"]

verona_players = ["Ngonge", "Folorunsho", "Djuric", "Bonazzoli", "Faraoni", "Montipo'"]

interesting_players_preasta = atalanta_players + bologna_players + cagliari_players + empoli_players + fiorentina_players + \
    frosinone_players + genoa_players + inter_players + juventus_players + lazio_players + \
        lecce_players + milan_players + monza_players + napoli_players + roma_players + \
            salernitana_players + sassuolo_players + torino_players + udinese_players + verona_players


In [26]:
preasta_players_2324_df = players_2324_df[np.isin(players_2324_df["Nome"], interesting_players_preasta)]
preasta_players_2223_df = players_2223_df[np.isin(players_2223_df["Nome"], interesting_players_preasta)]
preasta_stats_2223_df = stats_2223_df[np.isin(stats_2223_df["Nome"], interesting_players_preasta)]

In [27]:
print(preasta_players_2324_df.shape)
print(preasta_players_2223_df.shape)
print(preasta_stats_2223_df.shape)

(132, 13)
(100, 13)
(103, 18)


In [28]:
# three players did not finish the season in Serie A last year
# Viti -> Nice
# Malinovskyi -> Marseille
# Scamacca -> West Ham
preasta_stats_2223_df[np.invert(np.isin(preasta_stats_2223_df["Nome"], preasta_players_2223_df["Nome"]))]

,Id,R,Rm,Nome,Squadra,Pv,Mv,Fm,Gf,Gs,Rp,Rc,R+,R-,Ass,Amm,Esp,Au
166,5718,D,Dc,Viti,Empoli,0,0.0,0.00,0,0,0,0,0,0,0,0,0,0
352,4427,C,T,Malinovskyi,Atalanta,12,6.0,6.38,1,0,0,0,0,0,2,2,0,0
556,2137,A,Pc,Scamacca,Sassuolo,0,0.0,0.00,0,0,0,0,0,0,0,0,0,0


In [15]:
# players in Serie A who did not play last year in Serie A
preasta_players_2324_df[np.invert(np.isin(preasta_players_2324_df["Nome"], preasta_stats_2223_df["Nome"]))]

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
81,5067,D,E,Bakker,Atalanta,11,12,-1,10,11,-1,23,20
98,2640,D,Ds;Dc,Kolasinac,Atalanta,9,9,0,10,10,0,22,27
106,4317,D,Dc,N'dicka,Roma,8,9,-1,8,9,-1,18,18
117,4210,D,Dc,Mina,Fiorentina,7,8,-1,7,8,-1,12,12
124,5555,D,Ds;E,Kamara H.,Udinese,7,6,1,7,6,1,10,10
145,5365,D,Dd;Dc,Dragusin,Genoa,6,6,0,7,7,0,12,12
151,6202,D,Dc,Beukema,Bologna,6,6,0,6,6,0,22,22
226,6235,D,Dd;Ds;E,Kayode,Fiorentina,2,1,1,3,2,1,8,8
263,4856,C,W;A,Chukwueze,Milan,19,19,0,18,18,0,75,75
272,4142,C,C;T,Aouar,Roma,16,15,1,16,15,1,47,47


In [29]:
goalkeepers = Filter("R", "eq", "P").apply_to_df(preasta_players_2324_df)
defenders = Filter("R", "eq", "D").apply_to_df(preasta_players_2324_df)
midfielders = Filter("R", "eq", "C").apply_to_df(preasta_players_2324_df)
attackers = Filter("R", "eq", "A").apply_to_df(preasta_players_2324_df)
print(f"Number of goalkeepers: {goalkeepers.shape[0]:d}")
print(f"Number of defenders: {defenders.shape[0]:d}")
print(f"Number of midfielders: {midfielders.shape[0]:d}")
print(f"Number of attackers: {attackers.shape[0]:d}")

Number of goalkeepers: 7
Number of defenders: 36
Number of midfielders: 59
Number of attackers: 30


In [30]:
# example: midfielders with average > 6
Filter("R", "eq", "C").apply_to_df(Filter("Mv", "geq", 6).apply_to_df(preasta_stats_2223_df))

,Id,R,Rm,Nome,Squadra,Pv,Mv,Fm,Gf,Gs,Rp,Rc,R+,R-,Ass,Amm,Esp,Au
316,632,C,W;A,Zaccagni,Lazio,35,6.54,7.44,10,0,0,1,1,0,6,9,0,0
321,2085,C,T,Luis Alberto,Lazio,34,6.34,6.93,6,0,0,2,1,1,7,4,0,0
322,2167,C,W;A,Orsolini,Bologna,32,6.16,7.09,11,0,0,3,3,0,2,6,2,0
323,4486,C,W,Strefezza,Lecce,34,6.31,6.97,8,0,0,3,2,1,4,5,0,0
324,5685,C,C;T,Koopmeiners,Atalanta,33,6.36,7.14,10,0,0,4,2,2,5,7,0,0
325,1870,C,C,Barella,Inter,35,6.26,6.86,6,0,0,0,0,0,6,6,0,0
326,5687,C,W;T,Vlasic,Torino,34,6.22,6.82,5,0,0,0,0,0,6,1,0,0
329,332,C,W,Candreva,Salernitana,35,6.21,6.89,7,0,0,0,0,0,4,1,1,0
330,2848,C,C,Frattesi,Sassuolo,36,6.26,6.79,7,0,0,0,0,0,0,4,0,0
333,5858,C,C;T,Ferguson,Bologna,31,6.24,6.85,7,0,0,0,0,0,0,4,0,0


# ALBA fantacalcio - current teams